In [1]:
from dotenv import find_dotenv, load_dotenv
load_dotenv(find_dotenv())

True

### Agent With SearchAPI

In [2]:
from langchain.agents import AgentType, Tool, initialize_agent
from langchain_community.utilities import SearchApiAPIWrapper
from langchain.llms import OpenAI

In [ ]:
llm = OpenAI(temperature=0, model_name="gpt-4-turbo")
search = SearchApiAPIWrapper()

tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description="useful for when you need to ask with search",
    )
]

self_ask_with_search = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)
self_ask_with_search.run("قضیه اعتصاب طلافروشان تهران چیست؟")

### Search With DuckDuckGo

In [ ]:
from langchain_community.tools import DuckDuckGoSearchResults

search = DuckDuckGoSearchResults(backend="news")
print(search.run("امروز قیمت طلا چنده؟"))

### LLMChain with Tavily Search API

In [5]:
from langchain_community.retrievers import TavilySearchAPIRetriever

from langchain.callbacks.tracers import ConsoleCallbackHandler
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

In [ ]:
retriever = TavilySearchAPIRetriever(k=3)

prompt = ChatPromptTemplate.from_template(
"""Answer the question based only on the context provided.

Context: {context}

Question: {question}
"""
)

chain = (
    RunnablePassthrough.assign(context=(lambda x: x["question"]) | retriever)
    | prompt
    | ChatOpenAI(model="gpt-4-turbo", verbose=True)
    | StrOutputParser()
)

In [ ]:
chain.invoke(
    {"question": "الان قیمت طلا گرمی چنده؟"},
    config={'callbacks': [ConsoleCallbackHandler()]}
)